In [1]:
import sys
import gym
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import animation, rc
from IPython.display import Math, HTML
import os
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions.normal import Normal
import numpy as np
if not os.path.exists('.mujoco_setup_complete'):
  # Get the prereqs
  !apt-get -qq update
  !apt-get -qq install -y libosmesa6-dev libgl1-mesa-glx libglfw3 libgl1-mesa-dev libglew-dev patchelf
  # Get Mujoco
  !mkdir ~/.mujoco
  !wget -q https://mujoco.org/download/mujoco210-linux-x86_64.tar.gz -O mujoco.tar.gz
  !tar -zxf mujoco.tar.gz -C "$HOME/.mujoco"
  !rm mujoco.tar.gz
  # Add it to the actively loaded path and the bashrc path (these only do so much)
  !echo 'export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$HOME/.mujoco/mujoco210/bin' >> ~/.bashrc 
  !echo 'export LD_PRELOAD=$LD_PRELOAD:/usr/lib/x86_64-linux-gnu/libGLEW.so' >> ~/.bashrc 
  # THE ANNOYING ONE, FORCE IT INTO LDCONFIG SO WE ACTUALLY GET ACCESS TO IT THIS SESSION
  !echo "/root/.mujoco/mujoco210/bin" > /etc/ld.so.conf.d/mujoco_ld_lib_path.conf
  !ldconfig
  # Install Mujoco-py
  !pip3 install -U 'mujoco-py<2.2,>=2.1'
  # run once
  !touch .mujoco_setup_complete

try:
  if _mujoco_run_once:
    pass
except NameError:
  _mujoco_run_once = False
if not _mujoco_run_once:
  # Add it to the actively loaded path and the bashrc path (these only do so much)
  try:
    os.environ['LD_LIBRARY_PATH']=os.environ['LD_LIBRARY_PATH'] + ':/root/.mujoco/mujoco210/bin'
  except KeyError:
    os.environ['LD_LIBRARY_PATH']='/root/.mujoco/mujoco210/bin'
  try:
    os.environ['LD_PRELOAD']=os.environ['LD_PRELOAD'] + ':/usr/lib/x86_64-linux-gnu/libGLEW.so'
  except KeyError:
    os.environ['LD_PRELOAD']='/usr/lib/x86_64-linux-gnu/libGLEW.so'
  # presetup so we don't see output on first env initialization
  import mujoco_py
  _mujoco_run_once = True
#source of this code block : https://gist.github.com/BuildingAtom/3119ac9c595324c8001a7454f23bf8c8

Extracting templates from packages: 100%
(Reading database ... 109859 files and directories currently installed.)
Preparing to unpack .../00-libx11-6_2%3a1.6.9-2ubuntu1.5_amd64.deb ...
Unpacking libx11-6:amd64 (2:1.6.9-2ubuntu1.5) over (2:1.6.9-2ubuntu1.2) ...
Selecting previously unselected package libwayland-server0:amd64.
Preparing to unpack .../01-libwayland-server0_1.18.0-1ubuntu0.1_amd64.deb ...
Unpacking libwayland-server0:amd64 (1.18.0-1ubuntu0.1) ...
Selecting previously unselected package libgbm1:amd64.
Preparing to unpack .../02-libgbm1_21.2.6-0ubuntu0.1~20.04.2_amd64.deb ...
Unpacking libgbm1:amd64 (21.2.6-0ubuntu0.1~20.04.2) ...
Selecting previously unselected package libegl-mesa0:amd64.
Preparing to unpack .../03-libegl-mesa0_21.2.6-0ubuntu0.1~20.04.2_amd64.deb ...
Unpacking libegl-mesa0:amd64 (21.2.6-0ubuntu0.1~20.04.2) ...
Selecting previously unselected package libegl1:amd64.
Preparing to unpack .../04-libegl1_1.3.2-1~ubuntu0.20.04.2_amd64.deb ...
Unpacking libegl1:amd

In [2]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import gym
import matplotlib.pyplot as plt
import torch
import pdb
import copy
import random
!pip3 install box2d-py
!pip3 install gym[Box_2D]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [16 lines of output]
      Using setuptools (version 59.8.0).
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-3.10
      creating build/lib.linux-x86_64-3.10/Box2D
      copying library/Box2D/Box2D.py -> build/lib.linux-x86_64-3.10/Box2D
      copying library/Box2D/__init__.py -> build/lib.linux-x86_64-3.10/Box2D
      creating build/lib.linux-x86_64-3.10/Box2D/b2
      copying library/Box2D/b2/__init__.py -> build/lib.linux-x86_64-3.10/Box2D/b2
      running build_ext
      building 'Box2D._Box2D' extension
      swigging Box2D/Box2D.i to Box2D/Box2D_wrap.cpp
      swig -python -c++ -IBox2D -small -O -includeall -ignoremissing -w201 -globals b2Globals -outdir

In [3]:
class ReplayBuffer(object):
	def __init__(self, state_dim, action_dim, max_size=int(1e6)):
		self.max_size = max_size
		self.ptr = 0
		self.size = 0

		self.state = np.zeros((max_size, state_dim))
		self.action = np.zeros((max_size, action_dim))
		self.next_state = np.zeros((max_size, state_dim))
		self.reward = np.zeros((max_size, 1))
		self.not_done = np.zeros((max_size, 1))

		self.welford_state_n = 1
		self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

	def normalize_state(self, states, update=True):
		"""
		Use Welford's algorithm to normalize a state, and optionally update the statistics
		for normalizing states using the new state, online.
		"""
		states = torch.Tensor(states)
		states2 = states.data.clone()
		ii = 0
		for state in states:
			if self.welford_state_n == 1:
				self.welford_state_mean = torch.zeros(state.size(-1))
				self.welford_state_mean_diff = torch.ones(state.size(-1))

			if update:
				if len(state.size()) == 1: # if we get a single state vector
					state_old = self.welford_state_mean
					self.welford_state_mean += (state - state_old) / self.welford_state_n
					self.welford_state_mean_diff += (state - state_old) * (state - state_old)
					self.welford_state_n += 1
				else:
					raise RuntimeError # this really should not happen
			states2[ii] = (state - self.welford_state_mean) / np.sqrt(self.welford_state_mean_diff / self.welford_state_n)
			ii += 1
		return states2

	def add(self, state, action, next_state, reward, done):
		self.state[self.ptr] = state
		self.action[self.ptr] = action
		self.next_state[self.ptr] = next_state
		self.reward[self.ptr] = reward
		self.not_done[self.ptr] = 1. - done

		self.ptr = (self.ptr + 1) % self.max_size
		self.size = min(self.size + 1, self.max_size)


	def sample(self, batch_size):
		ind = np.random.randint(0,int(self.size), size=batch_size)
		return (
			torch.FloatTensor(self.state[ind]).to(self.device),
			#torch.FloatTensor(self.normalize_state(self.state[ind])).to(self.device),
			torch.FloatTensor(self.action[ind]).to(self.device),
			torch.FloatTensor(self.next_state[ind]).to(self.device),
			torch.FloatTensor(self.reward[ind]).to(self.device),
			torch.FloatTensor(self.not_done[ind]).to(self.device)
		)



def soft_update(target, source, tau):
	for target_param, param in zip(target.parameters(), source.parameters()):
		target_param.data.copy_(target_param.data * (1.0 - tau) + param.data * tau)

def hard_update(target, source):
	for target_param, param in zip(target.parameters(), source.parameters()):
		target_param.data.copy_(param.data)

In [4]:
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Implementation of Deep Deterministic Policy Gradients (DDPG)
# Paper: https://arxiv.org/abs/1509.02971
# [Not the implementation used in the TD3 paper]


class Actor(nn.Module):
	def __init__(self, state_dim, action_dim, max_action):
		super(Actor, self).__init__()

		self.l1 = nn.Linear(state_dim, 256)
		self.l2 = nn.Linear(256, 256)
		self.l3 = nn.Linear(256, action_dim)
		
		self.max_action = max_action

	
	def forward(self, state):
		a = F.relu(self.l1(state))
		a = F.relu(self.l2(a))
		return self.max_action * torch.tanh(self.l3(a))


class Critic(nn.Module):
	def __init__(self, state_dim, action_dim):
		super(Critic, self).__init__()

		self.l1 = nn.Linear(state_dim, 256)
		self.l2 = nn.Linear(256 + action_dim, 256)
		self.l3 = nn.Linear(256, 1)


	def forward(self, state, action):
		q = F.relu(self.l1(state))
		q = F.relu(self.l2(torch.cat([q, action], 1)))
		return self.l3(q)


class DDPG(object):
	def __init__(self, state_dim, action_dim, max_action, discount=0.99, tau=0.005):
		self.actor = Actor(state_dim, action_dim, max_action).to(device)
		self.actor_target = copy.deepcopy(self.actor)
		self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=3e-4)

		self.critic = Critic(state_dim, action_dim).to(device)
		self.critic_target = copy.deepcopy(self.critic)
		self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=3e-4)

		self.discount = discount
		self.tau = tau


	def select_action(self, state):
		state = torch.FloatTensor(state.reshape(1, -1)).to(device)
		return self.actor(state).cpu().data.numpy().flatten()


	def train(self, replay_buffer, batch_size=64):
		# Sample replay buffer 
		state, action, next_state, reward, not_done = replay_buffer.sample(batch_size)

		# Compute the target Q value
		target_Q = self.critic_target(next_state, self.actor_target(next_state))
		target_Q = reward + (not_done * self.discount * target_Q).detach()

		# Get current Q estimate
		current_Q = self.critic(state, action)

		# Compute critic loss
		critic_loss = F.mse_loss(current_Q, target_Q)

		# Optimize the critic
		self.critic_optimizer.zero_grad()
		critic_loss.backward()
		self.critic_optimizer.step()

		# Compute actor loss
		actor_loss = -self.critic(state, self.actor(state)).mean()
		
		# Optimize the actor 
		self.actor_optimizer.zero_grad()
		actor_loss.backward()
		self.actor_optimizer.step()

		# Update the frozen target models
		for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
			target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

		for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
			target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

In [5]:
import numpy as np
import torch
import gym
import argparse
import os
import copy
import pandas as pd
import json,os

alg="DDPG"
seed=3
envName='Ant-v3'
timestep_limit=1000000
evaluationStep=5000
printStep=50
initial_timestep=10000

def eval_policy(agent, enviroment_name,episodes=10):
    evaluation_env =gym.make(envName)
    average_reward = 0.
    for _ in range(episodes):
        state, _ = evaluation_env.reset()
        done = False
        truncuated=False
        while (not done) and (not truncuated):
            action = agent.select_action(np.array(state))
            state, reward, done, truncuated,_ = evaluation_env.step(action)
            average_reward += reward
    average_reward /= episodes
    return average_reward


env = gym.make(envName)
env.action_space.seed(seed)
T.manual_seed(seed)
np.random.seed(seed)

state_dim = env.observation_space.shape[0]
state_max = env.observation_space.shape
action_dim = env.action_space.shape[0]
max_action = float(env.action_space.high[0])

policy=DDPG(state_dim,action_dim,max_action,discount=0.99,tau=0.005)
variant = dict(algorithm=alg,env=envName)

replay_buffer = ReplayBuffer(state_dim, action_dim)

evaluations = [eval_policy(policy, envName)]

steps=0
while (steps <= timestep_limit):
    state, _ = env.reset(seed=seed)
    done, truncuated = False, False
    while (not done) and (not truncuated):
        if steps < initial_timestep:
            action = env.action_space.sample()
        else:
            action = (policy.select_action(np.array(state))+ np.random.normal(0, max_action * 0.1, size=action_dim)).clip(-max_action, max_action)
        state_, reward, done, truncuated, info = env.step(action)
        replay_buffer.add(state, action, state_, reward, done)
        steps += 1
        state = state_
        if steps>=initial_timestep:
            policy.train(replay_buffer, 100)
        if(steps%5000)==0:
            evaluation_reward = eval_policy(policy, envName)
            evaluations.append(evaluation_reward)
            print(f"Evaluation over {10} episodes: {evaluation_reward:.3f}  step{steps}")

if not os.path.exists(f"./data/{envName}/{alg}/seed{seed}"):
    os.makedirs(f'./data/{envName}/{alg}/seed{seed}')
with open(f'./data/{envName}/{alg}/seed{int(seed)}/variant.json', 'w') as outfile:
    json.dump(variant,outfile)
data = np.array(evaluations)
df = pd.DataFrame(data=data,columns=["Average Return"]).reset_index()
df['Timesteps'] = df['index'] * 2000
df['env'] = envName
df['algorithm_name'] = alg
df.to_csv(f'./data/{envName}/{alg}/seed{seed}/progress.csv', index = False)

/opt/conda/lib/python3.10/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment Ant-v3 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
/opt/conda/lib/python3.10/site-packages/gym/envs/mujoco/mujoco_env.py:190: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.warn(


Evaluation over 10 episodes: 998.407  step5000
Evaluation over 10 episodes: 998.992  step10000
Evaluation over 10 episodes: 770.456  step15000
Evaluation over 10 episodes: 653.680  step20000
Evaluation over 10 episodes: 767.537  step25000
Evaluation over 10 episodes: 44.807  step30000
Evaluation over 10 episodes: 72.671  step35000
Evaluation over 10 episodes: 703.542  step40000
Evaluation over 10 episodes: 207.415  step45000
Evaluation over 10 episodes: 158.505  step50000
Evaluation over 10 episodes: 478.333  step55000
Evaluation over 10 episodes: 325.786  step60000
Evaluation over 10 episodes: 421.109  step65000
Evaluation over 10 episodes: 226.066  step70000
Evaluation over 10 episodes: 709.216  step75000
Evaluation over 10 episodes: 482.426  step80000
Evaluation over 10 episodes: 428.217  step85000
Evaluation over 10 episodes: 548.550  step90000
Evaluation over 10 episodes: 422.880  step95000
Evaluation over 10 episodes: 512.381  step100000
Evaluation over 10 episodes: 460.051  step